In [1]:
import torch
import math

def create_model_matrix(translation, rotation, scale):
    tx, ty, tz = translation
    rx, ry, rz = rotation
    sx, sy, sz = scale
    
    # Translation Matrix
    T = torch.eye(4)
    T[0, 3] = tx
    T[1, 3] = ty
    T[2, 3] = tz
    
    # Rotation Matrices (around x, y, z axes)
    Rx = torch.tensor([
        [1, 0, 0, 0],
        [0, math.cos(rx), -math.sin(rx), 0],
        [0, math.sin(rx), math.cos(rx), 0],
        [0, 0, 0, 1]
    ])
    
    Ry = torch.tensor([
        [math.cos(ry), 0, math.sin(ry), 0],
        [0, 1, 0, 0],
        [-math.sin(ry), 0, math.cos(ry), 0],
        [0, 0, 0, 1]
    ])
    
    Rz = torch.tensor([
        [math.cos(rz), -math.sin(rz), 0, 0],
        [math.sin(rz), math.cos(rz), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    
    # Combined Rotation Matrix
    R = torch.matmul(torch.matmul(Rz, Ry), Rx)
    
    # Scaling Matrix
    S = torch.diag(torch.tensor([sx, sy, sz, 1.0]))
    
    # Model Matrix: M_model = T * R * S
    M_model = torch.matmul(torch.matmul(T, R), S)
    
    return M_model

# Example Usage:
translation = (2, 3, 4)
rotation = (0, math.radians(45), 0)  # Convert degrees to radians for y-axis
scale = (2, 2, 2)
M_model = create_model_matrix(translation, rotation, scale)
print("Model Matrix:\n", M_model)


Model Matrix:
 tensor([[ 1.4142,  0.0000,  1.4142,  2.0000],
        [ 0.0000,  2.0000,  0.0000,  3.0000],
        [-1.4142,  0.0000,  1.4142,  4.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])


In [8]:
def create_view_matrix(eye, target, up):
    eye = torch.tensor(eye,dtype = torch.float32)
    target = torch.tensor(target,dtype = torch.float32)
    up = torch.tensor(up,dtype = torch.float32)
    
    # Forward vector (normalized)
    f = (target - eye)
    f = f / torch.norm(f)
    
    # Right vector (normalized)
    r = torch.cross(f, up)
    r = r / torch.norm(r)
    
    # Up vector (recompute to ensure orthogonality)
    u = torch.cross(r, f)
    
    # View Matrix
    M_view = torch.eye(4)
    M_view[0, 0:3] = r
    M_view[1, 0:3] = u
    M_view[2, 0:3] = -f
    M_view[0, 3] = -torch.dot(r, eye)
    M_view[1, 3] = -torch.dot(u, eye)
    M_view[2, 3] = torch.dot(f, eye)
    
    return M_view

# Example Usage:
eye = (5, 5, 10)
target = (0, 0, 0)
up = (0, 1, 0)
M_view = create_view_matrix(eye, target, up)
print("View Matrix:\n", M_view)


View Matrix:
 tensor([[  0.8944,   0.0000,  -0.4472,  -0.0000],
        [ -0.1826,   0.9129,  -0.3651,  -0.0000],
        [  0.4082,   0.4082,   0.8165, -12.2474],
        [  0.0000,   0.0000,   0.0000,   1.0000]])


In [9]:
def create_projection_matrix(fov, aspect, near, far):
    f = 1.0 / math.tan(math.radians(fov) / 2.0)
    
    M_proj = torch.tensor([
        [f / aspect, 0, 0, 0],
        [0, f, 0, 0],
        [0, 0, (far + near) / (near - far), (2 * far * near) / (near - far)],
        [0, 0, -1, 0]
    ])
    
    return M_proj

# Example Usage:
fov = 60.0  # degrees
aspect_ratio = 16.0 / 9.0
near = 0.1
far = 100.0
M_proj = create_projection_matrix(fov, aspect_ratio, near, far)
print("Projection Matrix:\n", M_proj)


Projection Matrix:
 tensor([[ 0.9743,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.7321,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -1.0020, -0.2002],
        [ 0.0000,  0.0000, -1.0000,  0.0000]])


In [12]:
# Example vertex in local space (homogeneous coordinates)
v_local = torch.tensor([1.0, 1.0, 1.0, 1.0])  # (x, y, z, w)

# Model to World
v_world = torch.matmul(M_model, v_local)

# World to View
v_view = torch.matmul(M_view, v_world)

# View to Clip
v_clip = torch.matmul(M_proj, v_view)

print("Vertex in Clip Space:\n", v_clip)
print(f"Clip Cordinates to NIC Cordinates:{v_clip[:3]/v_clip[3]}")


Vertex in Clip Space:
 tensor([2.4648, 3.8490, 4.7788, 4.9690])
Clip Cordinates to NIC Cordinates:tensor([0.4960, 0.7746, 0.9617])
